# Clustering

We want to identify clusters/groups/classes in the data. Most of the time you would want to do that as part of an exploratory analysis.

It is more of a trial and error approach than a problem with a clearly defined solution. After all it is often used in situations were we do not have labels.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from sklearn.datasets import make_blobs

### 1. K-Means Clustering

0. Scale the data!
1. Initial centroids (randomly or from the data)
2. Assign datapoints to nearest centroid
3. calculate mean of all points off a cluster and set this as the new centroid
4. repeat until nothing changes anymore.


Visualization: https://stanford.edu/class/engr108/visualizations/kmeans/kmeans.html

In [ ]:
from sklearn.cluster import KMeans
# For big datasets: MiniBatchKMeans

In [ ]:
# Create a dataset with different clusters
X, y = make_blobs(n_features=2, n_samples=200, centers=5, random_state=42, cluster_std=1)

In [ ]:
# Plot the data
plt.scatter(X[:,0].astype(float), X[:, 1].astype(float))

In [ ]:
# Scale the data; MinMaxScaler scales data between 0 and 1
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

In [ ]:
Kmean = KMeans(n_clusters=5)
Kmean.fit(X)

In [ ]:
Kmean.cluster_centers_

In [ ]:
# Plot the data and draw the centroids of the clusters
plt.scatter(X[:,0].astype(float), X[:, 1].astype(float))
plt.scatter(Kmean.cluster_centers_[:,0],Kmean.cluster_centers_[:,1] )

In [ ]:
# plot with labels
fig, ax = plt.subplots()
scatter = ax.scatter(X[:,0].astype(float), X[:, 1].astype(float), c=Kmean.labels_)
legend1 = ax.legend(*scatter.legend_elements())
ax.add_artist(legend1)

In [ ]:
# Predicting an unseen point: 
test = np.array([0.5, 0.7]).reshape(1, -1)
Kmean.predict(test)

### 2. Evaluation

#### 2.1 Inertia

Sum of squared distances of samples to their closest cluster center.

In [ ]:
Kmean.inertia_

#### 2.2 Inertia + Elbow-method

In [ ]:
# Calculate the inertia for different numbers of clusters
inertia = []
for i in range(1,20):
    Kmean = KMeans(n_clusters=i)
    Kmean.fit(X)
    inertia.append(Kmean.inertia_)

In [ ]:
# Plot the inertia
plt.plot(inertia)

### 2.3 Silhouette Score

$$
\frac{b-a}{max(a, b)}
$$

where a is the mean intra-cluster distance and b is the mean nearest-cluster distance. It takes on values between -1 and 1, where 1 is the best: clusters are dense and well separated.

In [ ]:
from sklearn.metrics import silhouette_score

In [ ]:
Kmean = KMeans(n_clusters=4)
Kmean.fit(X)
print(f'The silhouette_score is {round(silhouette_score(X, Kmean.labels_), 2)}')

#### Advantages of K-means: 
    - fast
    - easy to understand
    
#### Disadvantages: 
    - non-deterministic
    - only works well for clusters of convex shape and similar size
    - we have to specify k
    - sensitive to outliers

### Curse of dimensionality: 
- euclidean distance works bad for high dimensional spaces (many features)
- all data points appear to be further from each other (increasing distance) the more dimensions we have, and with little distance variation -> bad for clustering. 

What is a possible solution?

### 3. When KMeans fails

In [ ]:
from sklearn.datasets import make_moons

In [ ]:
# Create the dataset
X_moons, _ = make_moons(1000, noise=0.12)

In [ ]:
# Scale the data
scaler = MinMaxScaler()
X_moons = scaler.fit_transform(X_moons)

In [ ]:
# Show the data
plt.scatter(X_moons[:,0], X_moons[:,1])
plt.show()

In [ ]:
# Cluster the data using Kmeans
Kmean_moons = KMeans(n_clusters=2)
Kmean_moons.fit(X_moons)

In [ ]:
# Show the result
plt.scatter(X_moons[:,0], X_moons[:,1], c=Kmean_moons.labels_)
plt.show()

https://scikit-learn.org/stable/modules/clustering.html#overview-of-clustering-methods

### 4. DBSCAN - Density based clustering

- creates clusters based on density -> clusters can have any shape
- no assumption about number of cluster
- hyperparamters: 
    - How close do points have to be together for a cluster (eps)
    - How many points constitute a cluster (min_samples)

**Algorithm:**

- Choose a random datapoint from the dataset
- Calculate the distance to each datapoint
- If there are are more then min_samples - 1 datapoints closer than eps from the datapoint, open a cluster and go to the next datapoint within the cluster; if not, mark the datapoint as outlier and go to the next datapoint
- Do the same for the next datapoint until all datapoints were visited

In [ ]:
from sklearn.cluster import DBSCAN

In [ ]:
# Fit the model
dbscan = DBSCAN(eps=0.04, min_samples=5)
labels = dbscan.fit_predict(X_moons)

In [ ]:
# Show the result
plt.scatter(X_moons[:,0].astype(float), X_moons[:, 1].astype(float), c=labels)

In [ ]:
silhouette_score(X_moons, labels)

#### 4.1 How to choose epsilon?

We could check out the typical closest distance between two datapoints in the set.

In [ ]:
from sklearn.neighbors import NearestNeighbors

In [ ]:
# Calculate the typical distance
neigh = NearestNeighbors(n_neighbors=2)
nbrs = neigh.fit(X_moons)
distances, indices = nbrs.kneighbors(X_moons)

In [ ]:
# Sort the distances
distances = np.sort(distances, axis=0)[:,1]

In [ ]:
# Plot the distances
plt.plot(distances)

### Advantages: 
- clusters can have any shape and size
- outliers are treated as such
- we don't have to specify number of clusters beforehand

### Disadvantages: 
- slower
- harder to tune
- harder to evaluate


### 5. Hierarchical Clustering (Agglomerative Clustering)
Builds a hierarchy from the bottom up. 

- Each datapoint starts as their own little cluster. 
- Then, a hierarchy is build through always aggregating the points that are closest together. 
- In the end, we have one big cluster, but can "cut" the tree of at each level to get specific number of clusters. 

In [ ]:
from scipy.cluster import hierarchy

X_2, _ = make_blobs(n_features=2, n_samples=200,centers=4, random_state=42, cluster_std=1.5)
agg = hierarchy.linkage(X_2)
d = hierarchy.dendrogram(agg, )

In [ ]:
plt.scatter(X_2[:,0], X_2[:,1])